In [30]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from sqlalchemy import create_engine
from langchain.agents import create_json_agent
from langchain_community.tools.json.tool import JsonSpec
from langchain_community.agent_toolkits import JsonToolkit

load_dotenv('../.env')


True

In [31]:
TABLE_NAME = os.getenv('EFORM_TABLE_NAME')
DB_NAME = os.getenv('EFORM_DB_NAME')
USER = "kamalleaf" #os.getenv('EFORM_DB_USER')
PASS = os.getenv('EFORM_DB_PASS')
HOST = os.getenv('EFORM_DB_HOST')
GPT_4 = "gpt-4-1106-preview"
GPT_3 = "gpt-3.5-turbo-0125"
LLM = ChatOpenAI(temperature=0, model=GPT_4)



In [6]:
ENGINE = create_engine(f"mysql+mysqlconnector://{USER}:{PASS}@{HOST}/{DB_NAME}")


In [7]:
sql_query = pd.read_sql_query(
    f"""
        SELECT *
        FROM {TABLE_NAME}
        WHERE entry_date > '2024-05-10'
        LIMIT 20
    """,
    ENGINE,
)

In [14]:
json_dicts = {}
for group_id, group_df in sql_query.groupby("id_group"):
    for _, row in group_df.iterrows():
        restricted = row["restricted"]
        form_title = row["form_title"].replace("/", "-")
        entry_date = int(row["entry_date"].timestamp())
        json_dict = json.loads(row.to_json())
        to_update = json.loads(json_dict.pop("data_components", "{}"))
        json_dict.update(to_update)
        json_name = f"{form_title}_{group_id}_{entry_date}"
        json_dicts[json_name] = json_dict
    

In [15]:
json_spec = JsonSpec(dict_=json_dicts, max_value_length=10000)
json_toolkit = JsonToolkit(spec=json_spec)

In [33]:
agent = create_json_agent(
    llm=LLM, toolkit=json_toolkit, verbose=True
)

In [34]:
agent.invoke("how many meetings are there?")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['Meeting Minutes_1_1715304598', 'Entry Permit For Real Estate Agent-Negotiator_1_1715563320', 'Meeting Minutes_1_1715569315', 'Meeting Minutes_1_1715658048', 'Walk-in Feedback Form_1_1715680998', 'Walk-in Feedback Form_1_1715680999', 'Vendor Company Registration Form_1_1715685420', 'Vendor Staff Registration Form_1_1715685667', 'Vendor Staff Registration Form_1_1715685770', 'Vendor Staff Registration Form_1_1715685871', 'Vendor Staff Registration Form_1_1715685893', 'Vendor Staff Registration Form_1_1715685917', 'Vendor Staff Registration Form_1_1715768604', 'Checklist_360_1715574634']
Thought:The keys that are prefixed with "Meeting Minutes" seem to be related to meetings. I should count how many keys start with "Meeting Minutes" to determine the number of meetings.
Final Answer: There are 3 meetings.

> Finished chain.


{'input': 'how many meetings are there?', 'output': 'There are 3 meetings.'}

In [35]:
agent.invoke("what are the meetings about?")




> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['Meeting Minutes_1_1715304598', 'Entry Permit For Real Estate Agent-Negotiator_1_1715563320', 'Meeting Minutes_1_1715569315', 'Meeting Minutes_1_1715658048', 'Walk-in Feedback Form_1_1715680998', 'Walk-in Feedback Form_1_1715680999', 'Vendor Company Registration Form_1_1715685420', 'Vendor Staff Registration Form_1_1715685667', 'Vendor Staff Registration Form_1_1715685770', 'Vendor Staff Registration Form_1_1715685871', 'Vendor Staff Registration Form_1_1715685893', 'Vendor Staff Registration Form_1_1715685917', 'Vendor Staff Registration Form_1_1715768604', 'Checklist_360_1715574634']
Thought:There are several keys related to meeting minutes which might contain information about what the meetings are about. I should explore one of these keys to find out more about the meeting topics.
Action: json_spec_list_keys
Action Input: data["Meeting Minutes_1_1715304598"]
Observation: ['id_digita

{'input': 'what are the meetings about?',
 'output': 'The meetings are about discussing the upcoming project plans and the allocation of resources.'}

In [41]:
agent.invoke("is there any affected area feedback by us")




> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['Meeting Minutes_1_1715304598', 'Entry Permit For Real Estate Agent-Negotiator_1_1715563320', 'Meeting Minutes_1_1715569315', 'Meeting Minutes_1_1715658048', 'Walk-in Feedback Form_1_1715680998', 'Walk-in Feedback Form_1_1715680999', 'Vendor Company Registration Form_1_1715685420', 'Vendor Staff Registration Form_1_1715685667', 'Vendor Staff Registration Form_1_1715685770', 'Vendor Staff Registration Form_1_1715685871', 'Vendor Staff Registration Form_1_1715685893', 'Vendor Staff Registration Form_1_1715685917', 'Vendor Staff Registration Form_1_1715768604', 'Checklist_360_1715574634']
Thought:The key that seems most relevant to the question about "affected area feedback" is likely to be one of the "Walk-in Feedback Form" keys, as feedback forms often contain information about areas affected by an issue or service.
Action: json_spec_list_keys
Action Input: data["Walk-in Feedback Form_1_

{'input': 'is there any affected area feedback by us',
 'output': 'Yes, there is affected area feedback. The feedback mentions "Block A Ground Floor Lobby" and specifically that the "Block A ground floor lobby ceiling light not functioning and dropped off already."'}